## テクスチャ特徴量抽出

In [39]:
import os, sys
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from texture_extractor import TextureExtractor

import logging
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR)

# データシートにて，データパスが書かれているカラムを設定
PATH_COLUMNS = "path"

### データシートの読み込み

In [40]:
dataseet = pd.read_csv("./sc_data_seet.csv", index_col=PATH_COLUMNS)
dataseet.head()

,f,k
path,,
images/gs_2700_5500.tiff,0.02700,0.055
images/gs_2352_5500.tiff,0.02352,0.055
images/gs_2865_5500.tiff,0.02865,0.055
images/gs_2016_5500.tiff,0.02016,0.055
images/gs_2511_5500.tiff,0.02511,0.055


### テクスチャ抽出インスタンス作成

In [41]:
texture_extractor = TextureExtractor()

### テクスチャ抽出の実行

In [42]:
df = pd.DataFrame()
for index, row in dataseet.iterrows():
    image = np.array( Image.open(index).convert('L') )
    image_array = np.array(image)
    mask_array = np.ones(image_array.shape)
    result = texture_extractor.get_texture_feature(image_array, mask_array)
    df = pd.concat([df, pd.DataFrame(result, index=[index])])

### テクスチャ特徴量と特性をマージしたデータフレーム作成

In [43]:
df = pd.merge(dataseet, df, left_index=True, right_index=True)

In [44]:
df.head()

,f,k,FO-10Percentile,FO-90Percentile,FO-Energy,FO-Entropy,FO-InterquartileRange,FO-Kurtosis,FO-Maximum,FO-MeanAbsoluteDeviation,...,GLDM-GrayLevelNonUniformity,GLDM-GrayLevelVariance,GLDM-HighGrayLevelEmphasis,GLDM-LargeDependenceEmphasis,GLDM-LargeDependenceHighGrayLevelEmphasis,GLDM-LargeDependenceLowGrayLevelEmphasis,GLDM-LowGrayLevelEmphasis,GLDM-SmallDependenceEmphasis,GLDM-SmallDependenceHighGrayLevelEmphasis,GLDM-SmallDependenceLowGrayLevelEmphasis
images/gs_2700_5500.tiff,0.02700,0.055,80.0,168.0,1.052519e+09,2.049164,67.0,1.522951,185.0,30.242184,...,17233.888977,1.657946,13.446045,41.562653,527.028015,6.426540,0.131398,0.041888,0.582785,0.004591
images/gs_2352_5500.tiff,0.02352,0.055,82.0,184.0,1.302279e+09,2.540300,70.0,1.757117,222.0,33.882882,...,11993.420715,2.466637,18.078156,34.740387,681.125763,5.074764,0.135526,0.047299,0.772729,0.005822
images/gs_2865_5500.tiff,0.02865,0.055,84.0,162.0,9.269245e+08,1.982162,51.0,2.015939,189.0,25.867374,...,18893.164276,1.444356,11.233932,43.314911,415.160843,7.799137,0.149927,0.034992,0.447770,0.004288
images/gs_2016_5500.tiff,0.02016,0.055,85.0,210.0,1.729606e+09,2.793457,71.0,2.160657,243.0,38.617567,...,10404.946899,3.439311,36.528137,35.857483,1520.027832,1.721780,0.050987,0.052151,1.552047,0.002785
images/gs_2511_5500.tiff,0.02511,0.055,80.0,174.0,1.149166e+09,2.281862,72.0,1.526665,202.0,32.400909,...,14778.194122,2.007081,23.545456,36.002777,792.861038,2.613772,0.063135,0.048760,1.168110,0.002719


In [45]:
texture_extractor.feature_class_func.keys()

dict_keys(['FO', 'GLCM', 'GLSZM', 'GLRLM', 'NGTDM', 'GLDM'])

In [46]:
target_key = "GLCM"
target_columns = list(filter(lambda item: target_key in item, df.columns))
df[target_columns].head()

,GLCM-Autocorrelation,GLCM-ClusterProminence,GLCM-ClusterShade,GLCM-ClusterTendency,GLCM-Contrast,GLCM-Correlation,GLCM-DifferenceAverage,GLCM-DifferenceEntropy,GLCM-DifferenceVariance,GLCM-Id,...,GLCM-Imc2,GLCM-InverseVariance,GLCM-JointAverage,GLCM-JointEnergy,GLCM-JointEntropy,GLCM-MCC,GLCM-MaximumProbability,GLCM-SumAverage,GLCM-SumEntropy,GLCM-SumSquares
images/gs_2700_5500.tiff,13.245484,59.214717,2.981083,6.238052,0.391724,0.881826,0.370994,0.991747,0.249277,0.817958,...,0.918151,0.352854,3.432769,0.158391,3.158651,0.884235,0.297216,6.865538,2.759654,1.657444
images/gs_2352_5500.tiff,17.835771,160.443047,-3.005072,9.388428,0.482138,0.902311,0.443701,1.049891,0.268375,0.784552,...,0.950979,0.410104,3.950848,0.085791,3.876865,0.903662,0.142105,7.901696,3.385640,2.467642
images/gs_2865_5500.tiff,11.064084,60.700265,7.998808,5.432392,0.345812,0.880306,0.337969,0.937529,0.228350,0.832323,...,0.914401,0.331106,3.129287,0.181242,3.051491,0.882739,0.366757,6.258575,2.695566,1.444551
images/gs_2016_5500.tiff,36.249147,385.407981,-20.761094,13.255047,0.509186,0.926016,0.448720,1.119552,0.301978,0.785691,...,0.967391,0.396067,5.750016,0.074280,4.193924,0.927171,0.165346,11.500032,3.662134,3.441058
images/gs_2511_5500.tiff,23.317141,92.404673,1.428589,7.563640,0.453740,0.886798,0.429244,1.034490,0.261572,0.789456,...,0.932657,0.407856,4.641084,0.119391,3.525315,0.891310,0.240883,9.282169,3.063849,2.004345
